In [ ]:
import warnings
warnings.simplefilter('always', DeprecationWarning)

from osgeo import gdal
from osgeo import ogr
from osgeo import osr
import numpy as np
import math
# from osgeo import gdal_array
# from osgeo import gdalnumeric

import os
import contextlib

gdal.UseExceptions()

def silent_remove(path):
    with contextlib.suppress(FileNotFoundError):
        os.remove(path)

## Lecture de l'ENC

In [ ]:
enc_ds = ogr.Open('data/ENC_ROOT/US4MA13M/US4MA13M.000')
assert enc_ds.GetDriver().GetDescription() == 'S57' # S57
m_covr_lyr = enc_ds.GetLayerByName('M_COVR')
soundg_lyr = enc_ds.GetLayerByName('SOUNDG')

covr_geo = [f.GetGeometryRef().Clone() for f in m_covr_lyr if f.GetField('CATCOV') == 1][0]
covr_geo

### Écriture de l'emprise dans `data/covr.geojson`

In [ ]:
json_drv = ogr.GetDriverByName('GeoJSON')
covr_path = 'data/covr.geojson'
def create_covr():
    silent_remove(covr_path)
    covr_ds = json_drv.CreateDataSource(covr_path)
    covr_lyr = covr_ds.CreateLayer('covr')
    covr_f = ogr.Feature(covr_lyr.GetLayerDefn())
    covr_f.SetGeometry(covr_geo)
    covr_lyr.CreateFeature(covr_f)
    covr_ds.Destroy()
create_covr()
del create_covr

## Lecture de la Bathy

### Réduction à la zone de l'ENC

In [ ]:
bathy_ds = gdal.Warp("data/bathy.tif", 'data/navd_bath_30m',
                        options = gdal.WarpOptions(
                                            format='GTiff', 
                                            cutlineDSName=covr_path,
                                            cutlineLayer='covr',
                                            cropToCutline=True,
                                            creationOptions=['COMPRESS=LZW'],
                                            dstNodata=np.nan))
bathy_ds.FlushCache()

In [ ]:
assert bathy_ds.RasterCount == 1
bathy_band = bathy_ds.GetRasterBand(1)
bathy_np = bathy_band.ReadAsArray()
bathy_band.GetNoDataValue()

In [ ]:
bathy_np.shape # Northing, Easting

In [ ]:
# b = gdal.Open('data/navd_bath_30m')
# b = gdal.Open('data/bathy.tif')
b = bathy_ds
geotransform = b.GetGeoTransform()
print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))
print("???        = ({}, {})".format(geotransform[2], geotransform[4]))

In [ ]:
# b = gdal.Open('data/navd_bath_30m')
b = gdal.Open('data/bathy.tif')
# b = bathy_ds
geotransform = b.GetGeoTransform()
print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))
print("???        = ({}, {})".format(geotransform[2], geotransform[4]))

In [ ]:
geotransform

In [ ]:
assert geotransform[2] == 0
assert geotransform[4] == 0
def z(L, P): # small matrix, faster than numpy
    Xp = geotransform[0] + (P+.5)*geotransform[1] 
    Yp = geotransform[3] + (L+.5)*geotransform[5]
    return Xp, Yp

def antiz(Xp, Yp):
    P = math.floor((Xp - geotransform[0]) / geotransform[1])
    L = math.floor((Yp - geotransform[3]) / geotransform[5])
    return L, P

In [ ]:
z(0,0), z(2000,0), z(0, 2000)

In [ ]:
antiz(328133.590626, 4722220.06), antiz(388133.590626, 4722220.06), antiz(328133.590626, 4662220.06)

In [ ]:
antiz(328148.590626, 4722205.06), antiz(328148.590626, 4662205.06), antiz(388148.590626, 4722205.06)

In [ ]:
bathy_np[antiz(415283.0,4717460.3)]